<a href="https://colab.research.google.com/github/overfit-ir/persian-twitter-ner/blob/master/tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip -q install transformers
! pip -q install parsivar

     |████████████████████████████████| 1.1MB 7.3MB/s 
     |████████████████████████████████| 890kB 41.6MB/s 
     |████████████████████████████████| 3.0MB 40.4MB/s 
     |████████████████████████████████| 1.1MB 40.6MB/s 
     |████████████████████████████████| 36.2MB 91kB/s 
     |████████████████████████████████| 1.5MB 40.6MB/s 


In [3]:
import torch
from __future__ import unicode_literals
# from hazm import *
from parsivar import *
from pprint import pprint
import torch
import numpy as np
import pandas as pd
from datetime import datetime
from scipy import spatial
from transformers import AutoConfig, AutoTokenizer, AutoModel, AutoModelForTokenClassification
parsivar_tokenizer = Tokenizer()

In [5]:
# config = AutoConfig.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
# tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-ner-uncased")
# model = AutoModel.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
model = AutoModelForTokenClassification.from_pretrained("HooshvareLab/bert-base-parsbert-ner-uncased")
model.eval()
# !mkdir ./model
# model.save_pretrained('./model/')
# tokenizer.save_pretrained('./model/')
# config.save_pretrained('./model/')

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(100000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [6]:
tweet = """ محسن رضایی هم یک جوان ۲۶،۲۷ ساله بوده
مدیریت جنگ با مدیریت فعلی کشور که اکثرا افراد لیبرابی هستن که در جنگ هیچ نقشی نداشتن و یا نهایتا پشت جبهه مشغول بودن قابل مقایسه نیست
"""
print(tweet)

 محسن رضایی هم یک جوان ۲۶،۲۷ ساله بوده
مدیریت جنگ با مدیریت فعلی کشور که اکثرا افراد لیبرابی هستن که در جنگ هیچ نقشی نداشتن و یا نهایتا پشت جبهه مشغول بودن قابل مقایسه نیست



In [ ]:
# input_sentence = torch.tensor(tokenizer.encode(tweet, truncation=True, max_length = 500)).unsqueeze(0)
# print(input_sentence)
# out = model(input_sentence)
# print(out)

In [17]:
input_sentence = tokenizer.encode(tweet, return_tensors="pt", truncation=True, max_length = 500)
print(input_sentence)
out = model(input_sentence)
print(out)

tensor([[    2,  5403,  6993,  2063,  2076,  2934,  3649,   300,  3566,  3503,
          2461,  2856,  2941,  2037,  2856,  4293,  2116,  2046, 13895,  2698,
         10321,  2697, 25594,  2046,  2028,  2941,  2608,  8105,  8244,   331,
          2177,  7984,  3692,  5117,  4759,  3155,  2773,  4235,  2380,     4]])
(tensor([[[-6.9182e-01, -2.4831e+00, -2.0998e+00,  4.7954e-01, -1.9198e+00,
           2.3032e-01, -3.0040e+00,  2.9702e+00, -1.2548e+00, -2.1010e+00,
          -7.5496e-01, -2.0688e+00, -2.2385e+00, -1.5329e+00, -9.8617e-01,
           3.4783e-01, -2.2564e+00, -4.9431e-01, -1.8306e+00, -2.1018e+00,
           1.2600e+01],
         [ 2.1841e-01, -1.3621e+00, -1.9072e+00,  1.2404e+00, -1.1196e+00,
          -6.3436e-02, -5.2908e-01,  1.1838e+01, -1.4717e+00, -1.1050e+00,
          -1.8856e+00, -5.2482e-01, -1.5222e+00, -1.4841e+00, -1.3268e+00,
          -3.5249e-01, -1.1725e+00,  9.4828e-01, -1.8559e+00, -1.7725e+00,
          -1.3642e-01],
         [-1.3521e+00, -1.7598e+0